![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)
 
## Create Azure Kubernetes Service cluster and deploy SQL Server 2019 CTP 3.2 big data cluster
 
This notebook walks through the process of creating a new Azure Kubernetes Service cluster first, and then deploys a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 CTP 3.2 big data cluster</a> on the newly created AKS cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* The **Required information** cell will prompt you for a password that will be used to access the cluster controller, SQL Server, and Knox.
* The values in the **Azure settings** and **Default settings** cell can be changed as appropriate.

### **Prerequisites**
Ensure the following tools are installed and added to PATH before proceeding.

|Tools|Description|Installation|
|---|---|---|
| Azure CLI |Command-line tool for managing Azure services. Used to create AKS cluster | [Installation](https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest) |
|kubectl | Command-line tool for monitoring the underlying Kuberentes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |
|azdata | Command-line tool for installing and managing a big data cluster |[Installation](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-azdata?view=sqlallproducts-allversions) |

### **Check dependencies**

In [1]:
import sys
def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{cmd}\n')
    print(f'Successfully executed: {cmd}')

cmd = 'az --version'
run_command()
cmd = 'kubectl version --client=true'
run_command()
cmd = 'azdata --version'
run_command()

### **Required information**

In [2]:
import getpass
mssql_password = getpass.getpass(prompt = 'SQL Server 2019 big data cluster controller password')
if mssql_password == "":
    sys.exit(f'Password is required')
confirm_password = getpass.getpass(prompt = 'Confirm password')
if mssql_password != confirm_password:
    sys.exit(f'Passwords do not match.')
print('Password accepted, you can also use the same password to access Knox and SQL Server.')

### **Azure settings**
*Subscription ID*: visit <a href="https://portal.azure.com/#blade/Microsoft_Azure_Billing/SubscriptionsBlade">here</a> to find out the subscriptions you can use, if you leave it unspecified, the default subscription will be used.

*VM Size*: visit <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/linux/sizes">here</a> to find out the available VM sizes you could use. 
 
*Region*: visit <a href="https://azure.microsoft.com/en-us/global-infrastructure/services/?products=kubernetes-service">here</a> to find out the Azure regions where the Azure Kubernettes Service is available.

In [3]:
azure_subscription_id = ""
azure_vm_size = "Standard_E4s_v3"
azure_region = "eastus"
azure_vm_count = int(5)

### **Default settings**

In [4]:
import time
mssql_cluster_name = 'mssql-cluster'
mssql_controller_username = 'admin'
azure_resource_group = mssql_cluster_name + '-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
aks_cluster_name = azure_resource_group
configuration_profile = 'aks-dev-test'
configuration_folder = 'mssql-bdc-configuration'
print(f'Azure subscription: {azure_subscription_id}')
print(f'Azure VM size: {azure_vm_size}')
print(f'Azure VM count: {str(azure_vm_count)}')
print(f'Azure region: {azure_region}')
print(f'Azure resource group: {azure_resource_group}')
print(f'AKS cluster name: {aks_cluster_name}')
print(f'SQL Server big data cluster name: {mssql_cluster_name}')
print(f'SQL Server big data cluster controller user name: {mssql_controller_username}')
print(f'Deployment configuration profile: {configuration_profile}')
print(f'Deployment configuration: {configuration_folder}')

### **Login to Azure**

This will open a web browser window to enable credentials to be entered. If this cells is hanging forever, it might be because your Web browser windows is waiting for you to enter your Azure credentials!


In [5]:
cmd = f'az login'
run_command()


### **Set active Azure subscription**

In [6]:
if azure_subscription_id != "":
    cmd = f'az account set --subscription {azure_subscription_id}'
    run_command()
else:
    print('Using the default Azure subscription', {azure_subscription_id})
cmd = f'az account show'
run_command()

### **Create Azure resource group**

In [7]:
cmd = f'az group create --name {azure_resource_group} --location {azure_region}'
run_command()

### **Create AKS cluster**

In [8]:
cmd = f'az aks create --name {aks_cluster_name} --resource-group {azure_resource_group} --generate-ssh-keys --node-vm-size {azure_vm_size} --node-count {azure_vm_count}' 
run_command()

### **Set the new AKS cluster as current context**

In [9]:
cmd = f'az aks get-credentials --resource-group {azure_resource_group} --name {aks_cluster_name} --admin --overwrite-existing'
run_command()

### **Create a deployment configuration file**

In [10]:
import os
os.environ["ACCEPT_EULA"] = 'yes'
cmd = f'azdata bdc config init --source {configuration_profile} --target {configuration_folder} --force'
run_command()
cmd = f'azdata bdc config replace -c {configuration_folder}/cluster.json -j metadata.name={mssql_cluster_name}'
run_command()

### **Create SQL Server 2019 big data cluster**

In [11]:
print (f'Creating SQL Server 2019 big data cluster: {mssql_cluster_name} using configuration {configuration_folder}')
os.environ["CONTROLLER_USERNAME"] = mssql_controller_username
os.environ["CONTROLLER_PASSWORD"] = mssql_password
os.environ["MSSQL_SA_PASSWORD"] = mssql_password
os.environ["KNOX_PASSWORD"] = mssql_password
cmd = f'azdata bdc create -c {configuration_folder}'
run_command()

### **Login to SQL Server 2019 big data cluster**

In [12]:
cmd = f'azdata login --cluster-name {mssql_cluster_name}'
run_command()

### **Show SQL Server 2019 big data cluster endpoints**

In [13]:
import json,html,pandas
from IPython.display import *
pandas.set_option('display.max_colwidth', -1)
cmd = f'azdata bdc endpoint list'
cmdOutput = !{cmd}
endpoints = json.loads(''.join(cmdOutput))
endpointsDataFrame = pandas.DataFrame(endpoints)
endpointsDataFrame.columns = [' '.join(word[0].upper() + word[1:] for word in columnName.split()) for columnName in endpoints[0].keys()]
display(HTML(endpointsDataFrame.to_html(index=False, render_links=True)))

### **Connect to master SQL Server instance in Azure Data Studio**
Click the link below to connect to the master SQL Server instance of the SQL Server 2019 big data cluster.

In [14]:
sqlEndpoints = [x for x in endpoints if x['name'] == 'sql-server-master']
if sqlEndpoints and len(sqlEndpoints) == 1:
    connectionParameter = '{"serverName":"' + sqlEndpoints[0]['endpoint'] + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(mssql_password) + '}'
    display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to master SQL Server instance</font></a><br/>'))
else:
    sys.exit('Could not find the master SQL Server instance endpoint')